In [1]:
import keras
from keras import layers

In [3]:
inputs = keras.Input(shape=(28,28,1))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs = inputs, outputs = outputs)

In [4]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 28, 28, 1)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 26, 26, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 13, 13, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 11, 11, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 5, 5, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 3, 3, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1152)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 10)                  │          11,530 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 104,202 (407.04 KB)

 Trainable params: 104,202 (407.04 KB)

 Non-trainable params: 0 (0.00 B)

#### Explanation:

<ol>
  <li>CNNs expect input tensors of shape <code>(image_height, image_width, image_channels)</code> (excluding the batch dimension).</li>
  <li>For MNIST, the input shape is configured as <code>(28, 28, 1)</code>.</li>
  <li>The output of each <code>Conv2D</code> and <code>MaxPooling2D</code> layer is a rank-3 tensor with shape <code>(height, width, channels)</code>.</li>
  <li>The width and height of feature maps typically decrease as the network goes deeper.</li>
  <li>The number of output channels in <code>Conv2D</code> layers is determined by the first argument (e.g., 32, 64, or 128).</li>
  <li>After the final <code>Conv2D</code> layer in the example, the output shape is <code>(3, 3, 128)</code>.</li>
  <li>Densely connected (<code>Dense</code>) layers, used for classification, require 1D vector inputs.</li>
  <li>A <code>Flatten</code> layer is used to convert the 3D output of the convolutional part into a 1D vector before feeding it to the <code>Dense</code> layers.</li>
  <li>The final <code>Dense</code> layer for 10-way classification (like MNIST digits) has 10 output units and a <code>softmax</code> activation function.</li>
</ol>

In [7]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype("float32") / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype("float32") / 255

model.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.8798 - loss: 0.3839
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.9851 - loss: 0.0477
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.9904 - loss: 0.0305
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.9924 - loss: 0.0247
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.9957 - loss: 0.0152


#### Lets evaluate model on the test data

In [8]:
test_loss, test_accuracy = model.evaluate(test_images, test_labels)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9892 - loss: 0.0341


#### Densely connected model from chapter 2 had a test accuracy of 97.8%, the basic convnet has a test accuracy of 98.92%: we decreased the error rate by about 55% (relative)

#### Reason why convents are more effective then Dense layers?

<ol>
  <li><strong>Dense layers learn global patterns</strong> across the entire input feature space (e.g., all pixels in an MNIST digit).</li>
  <li><strong>Convolution layers learn local patterns</strong> within small 2D windows of the input (e.g., 3x3 patches in an image).</li>
  <li>This local learning gives CNNs two key properties:
    <ol type="a">
      <li><strong>Translation Invariance:</strong> Once a pattern is learned in one location, the convnet can recognize it in any other location without relearning. Dense layers would need to learn the same pattern anew at each location. This makes CNNs data-efficient for image processing.</li>
      <li><strong>Spatial Hierarchies of Patterns:</strong> CNNs learn increasingly complex patterns through layers. The first layer learns small local features (like edges), the second learns larger patterns from these features, and so on. This efficient learning of abstract visual concepts is possible because the visual world has a spatial hierarchy.</li>
    </ol>
  </li>
</ol>

<ol>
  <li>Convolutions operate on rank-3 tensors called feature maps with spatial axes (height, width) and a depth axis (channels).</li>
  <li>For RGB images, the depth is 3 (red, green, blue). Black-and-white (e.g., MNIST) has a depth of 1 (grayscale).</li>
  <li>The convolution extracts patches from the input feature map and applies the same transformation to produce an output feature map (still rank-3 with height and width).</li>
  <li>The output depth is arbitrary and represents filters, encoding specific aspects of the input data.</li>
  <li>In the MNIST example, a (28, 28, 1) input becomes (26, 26, 32) after the first convolution (32 filters).</li>
  <li>Convolutions are defined by two key parameters:
    <ol type="a">
      <li><strong>Patch size:</strong> Typically 3x3 or 5x5 (3x3 in the example).</li>
      <li><strong>Output depth:</strong> The number of filters computed.</li>
    </ol>
  </li>
  <li>In Keras <code>Conv2D</code> layers, these are specified as <code>Conv2D(output_depth, (window_height, window_width))</code>.</li>
  <li>A convolution works by sliding windows (e.g., 3x3 or 5x5) over the 3D input feature map.</li>
  <li>At each location, a 3D patch of surrounding features (shape: window height, window width, input depth) is extracted.</li>
  <li>Each 3D patch is transformed into a 1D vector (shape: output depth) via a tensor product with a learned weight matrix called the convolution kernel (reused across patches).</li>
  <li>These 1D vectors are then spatially reassembled into a 3D output map (shape: height, width, output depth).</li>
  <li>Each spatial location in the output corresponds to the same location in the input.</li>
</ol>